In [5]:
import numpy as np
import pandas as pd
import os
import numpy as np
import pandas as pd
import os
from gensim.test.utils import common_texts
from gensim.models import Word2Vec



In [17]:

dataset= pd.read_csv("./news_articles.csv")
dataset

,docno,text
0,FBIS3-40266,languag romanian articl typecso text draft law...
1,FBIS3-40267,languag romanian articl typecso articl stelian...
2,FBIS3-40268,languag slovak articl typebfn commentari juliu...
3,FBIS3-40269,languag slovak articl typecso interview pavol ...
4,FBIS3-40270,languag slovak articl typebfn ta sr report ste...
...,...,...
414950,LA040890-0022,regard april 1 pop eye report film secur guard...
414951,LA040890-0023,mark leviton whose march 25 letter respond dvo...
414952,LA040890-0024,true capit demand human interact conduct volun...
414953,LA040890-0025,with paint face depict classic work art van go...


In [18]:
%%time
dataset['text_list']= dataset['text'].apply(lambda x: [item for item in str(x).split()])
dataset

CPU times: user 10.1 s, sys: 25.2 s, total: 35.3 s
Wall time: 52.4 s


,docno,text,text_list
0,FBIS3-40266,languag romanian articl typecso text draft law...,"[languag, romanian, articl, typecso, text, dra..."
1,FBIS3-40267,languag romanian articl typecso articl stelian...,"[languag, romanian, articl, typecso, articl, s..."
2,FBIS3-40268,languag slovak articl typebfn commentari juliu...,"[languag, slovak, articl, typebfn, commentari,..."
3,FBIS3-40269,languag slovak articl typecso interview pavol ...,"[languag, slovak, articl, typecso, interview, ..."
4,FBIS3-40270,languag slovak articl typebfn ta sr report ste...,"[languag, slovak, articl, typebfn, ta, sr, rep..."
...,...,...,...
414950,LA040890-0022,regard april 1 pop eye report film secur guard...,"[regard, april, 1, pop, eye, report, film, sec..."
414951,LA040890-0023,mark leviton whose march 25 letter respond dvo...,"[mark, leviton, whose, march, 25, letter, resp..."
414952,LA040890-0024,true capit demand human interact conduct volun...,"[true, capit, demand, human, interact, conduct..."
414953,LA040890-0025,with paint face depict classic work art van go...,"[with, paint, face, depict, classic, work, art..."


In [14]:
%%time
model = Word2Vec(sentences=dataset['text_list'], vector_size=100, window=5, min_count=1, workers=4)
model.save("word2vec.model")
model = Word2Vec.load("word2vec.model")
#model.train([["hello", "world"]], total_examples=1, epochs=1)
model.init_sims(replace = True)

<timed exec>:5: DeprecationWarning: Call to deprecated `init_sims` (Gensim 4.0.0 implemented internal optimizations that make calls to init_sims() unnecessary. init_sims() is now obsoleted and will be completely removed in future versions. See https://github.com/RaRe-Technologies/gensim/wiki/Migrating-from-Gensim-3.x-to-4).


CPU times: user 24min 7s, sys: 1min 3s, total: 25min 10s
Wall time: 7min 49s


In [15]:
%%time


def vectorize(list_of_docs, model):
    
    features = []

    for tokens in list_of_docs:
        zero_vector = np.zeros(model.vector_size)
        vectors = []
        for token in tokens:
            if token in model.wv:
                try:
                    vectors.append(model.wv[token])
                except KeyError:
                    continue
        if vectors:
            vectors = np.asarray(vectors)
            avg_vec = vectors.mean(axis=0)
            features.append(avg_vec)
        else:
            features.append(zero_vector)
    return features
    
vectorized_docs = vectorize(dataset['text_list'], model=model)
len(vectorized_docs), len(vectorized_docs[0])

CPU times: user 2min 13s, sys: 3.12 s, total: 2min 16s
Wall time: 2min 18s


(414955, 100)

In [13]:
vectorized_docs[0]


array([-0.01912336,  0.04229745, -0.01038675,  0.02634132, -0.00790241,
        0.01080316, -0.07943522, -0.05295208,  0.05602971, -0.01650112,
        0.02788198,  0.02973842, -0.04615209, -0.01225102,  0.01900151,
        0.01256215, -0.02570641,  0.05610185, -0.01466871,  0.09858072,
       -0.10927626,  0.04409878, -0.09178817,  0.0260084 ,  0.00523009,
        0.01851544, -0.04831962,  0.06036098, -0.04803628, -0.00669505,
       -0.00459618, -0.05864758,  0.01148522,  0.01205573, -0.00227706,
       -0.01262612, -0.03145257,  0.03344459, -0.02208221,  0.0250313 ,
       -0.00757672,  0.05617931, -0.0494312 ,  0.02360131, -0.06967851,
        0.0028914 , -0.03064244,  0.012432  , -0.00311508, -0.05991148,
        0.00866682, -0.02002352,  0.03189332,  0.00754828,  0.01880636,
       -0.10264756, -0.00643405, -0.04900251,  0.02248738,  0.02934483,
        0.02917347,  0.05195703, -0.02530161, -0.01312673,  0.00431714,
       -0.03227659, -0.00261637, -0.01095331,  0.06089908, -0.02